<a href="https://colab.research.google.com/github/chemvatho/Peekbank-Analysis/blob/main/Peekbank_Interactive_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🔬 Peekbank: Children's Word Recognition Analysis

## Interactive Visual Analytics Dashboard

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/YOUR_USERNAME/peekbank-analysis/blob/main/Peekbank_Interactive_Analysis.ipynb)

---

In [1]:
# Install packages
!pip install mysql-connector-python pandas numpy scipy -q
!pip install -U plotly kaleido -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.1/34.1 MB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 112.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.0/69.0 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.5 MB/s eta 0:00:00


In [2]:
# Install system dependencies required by Kaleido
!sudo apt update && sudo apt-get install -y libnss3 libatk-bridge2.0-0 libcups2 libxcomposite1 libxdamage1 libxfixes3 libxrandr2 libgbm1 libxkbcommon0 libpango-1.0-0 libcairo2 libasound2

print("✓ System dependencies for Kaleido installed.")

Get:1 https://cli.github.com/packages stable InRelease [3,917 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [83.8 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://cli.github.com/packages stable/main amd64 Packages [356 B]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease [24.6 kB]
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,887 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,673 kB]
Get:14 http://security.u

In [3]:
import mysql.connector
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
import importlib # Import importlib
from scipy import stats
from scipy.ndimage import gaussian_filter1d
import warnings
import os

# Import kaleido and ensure its Chromium dependency is available
import kaleido
kaleido.get_chrome_sync()

warnings.filterwarnings('ignore')

# Reload plotly.io to ensure kaleido is detected if installed after initial import
importlib.reload(pio)

# Set Plotly template for beautiful plots
pio.templates.default = "plotly_white"

# Custom color palettes
COLORS = {
    'primary': '#3498DB',
    'secondary': '#2ECC71',
    'accent': '#E74C3C',
    'warning': '#F39C12',
    'dark': '#2C3E50',
    'light': '#ECF0F1',
    'purple': '#9B59B6',
    'teal': '#1ABC9C'
}

# Age group color palette (colorblind-friendly)
AGE_COLORS = {
    '6-12mo': '#E63946',
    '12-18mo': '#F4A261',
    '18-24mo': '#E9C46A',
    '24-36mo': '#2A9D8F',
    '36-48mo': '#264653',
    '48+mo': '#7209B7'
}

print("✓ Setup complete! Ready for beautiful visualizations 🎨")

✓ Setup complete! Ready for beautiful visualizations 🎨


In [4]:
# Connect to Peekbank
conn = mysql.connector.connect(
    host='34.210.173.143',
    port=3306,
    database='2025.1',
    user='reader',
    password='gazeofraccoons'
)
print("✓ Connected to Peekbank database!")

# Create data directory
os.makedirs('peekbank_data', exist_ok=True)

✓ Connected to Peekbank database!


In [5]:
# Download data
print("📥 Downloading data...")

# Metadata
datasets = pd.read_sql("SELECT * FROM datasets", conn)
subjects = pd.read_sql("SELECT * FROM subjects", conn)
stimuli = pd.read_sql("SELECT * FROM stimuli", conn)
administrations = pd.read_sql("SELECT * FROM administrations", conn)

# Eye-tracking data with age
aoi_query = """
SELECT aoi.*, adm.age, adm.subject_id, adm.dataset_id
FROM aoi_timepoints aoi
JOIN administrations adm ON aoi.administration_id = adm.administration_id
WHERE adm.age IS NOT NULL
LIMIT 1500000
"""
aoi_data = pd.read_sql(aoi_query, conn)

# Word info
trial_words = pd.read_sql("""
SELECT t.trial_id, s.english_stimulus_label as target_word
FROM trials t
JOIN trial_types tt ON t.trial_type_id = tt.trial_type_id
JOIN stimuli s ON tt.target_id = s.stimulus_id
""", conn)

print(f"✓ Downloaded {len(aoi_data):,} eye-tracking timepoints")
print(f"✓ {len(datasets)} datasets, {len(subjects):,} subjects")

📥 Downloading data...
✓ Downloaded 1,500,000 eye-tracking timepoints
✓ 39 datasets, 3,537 subjects


In [6]:
import os

save_dir = "exported_data"
os.makedirs(save_dir, exist_ok=True)

datasets.to_csv(f"{save_dir}/datasets.csv", index=False)
subjects.to_csv(f"{save_dir}/subjects.csv", index=False)
stimuli.to_csv(f"{save_dir}/stimuli.csv", index=False)
administrations.to_csv(f"{save_dir}/administrations.csv", index=False)
aoi_data.to_csv(f"{save_dir}/aoi_timepoints.csv", index=False)
trial_words.to_csv(f"{save_dir}/trial_words.csv", index=False)

print("✅ All datasets saved as CSV")


✅ All datasets saved as CSV


In [7]:
# Prepare data
looking_data = aoi_data.copy()
looking_data['target_looking'] = (looking_data['aoi'] == 'target').astype(float)
looking_data['distractor_looking'] = (looking_data['aoi'] == 'distractor').astype(float)
looking_data['valid_look'] = looking_data['aoi'].isin(['target', 'distractor'])
looking_data['prop_target'] = np.where(looking_data['valid_look'], looking_data['target_looking'], np.nan)

looking_data['age_group'] = pd.cut(
    looking_data['age'],
    bins=[0, 12, 18, 24, 36, 48, 100],
    labels=['6-12mo', '12-18mo', '18-24mo', '24-36mo', '36-48mo', '48+mo']
)

# Merge with word info
looking_data = looking_data.merge(trial_words, on='trial_id', how='left')

print(f"✓ Prepared {len(looking_data):,} timepoints for analysis")

✓ Prepared 1,500,000 timepoints for analysis


---

# 📊 Interactive Visualizations

---

## 1. Time Course of Word Recognition

This animated visualization shows how children's looking behavior unfolds over time after hearing a word.

In [8]:
# Compute time course
tc_data = looking_data[
    (looking_data['t_norm'] >= -500) &
    (looking_data['t_norm'] <= 2000)
].copy()

time_course = tc_data.groupby('t_norm').agg({
    'prop_target': ['mean', 'std', 'count']
}).reset_index()
time_course.columns = ['time', 'mean', 'std', 'n']
time_course['se'] = time_course['std'] / np.sqrt(time_course['n'])
time_course['ci_lower'] = time_course['mean'] - 1.96 * time_course['se']
time_course['ci_upper'] = time_course['mean'] + 1.96 * time_course['se']

# Smooth the data
time_course['mean_smooth'] = gaussian_filter1d(time_course['mean'], sigma=2)

# Re-import plotly.io to ensure kaleido is detected after installation
import plotly.io as pio

# Create figure
fig = go.Figure()

# Confidence interval band
fig.add_trace(go.Scatter(
    x=list(time_course['time']) + list(time_course['time'][::-1]),
    y=list(time_course['ci_upper']) + list(time_course['ci_lower'][::-1]),
    fill='toself',
    fillcolor='rgba(52, 152, 219, 0.2)',
    line=dict(color='rgba(255,255,255,0)'),
    name='95% CI',
    showlegend=True,
    hoverinfo='skip'
))

# Main line
fig.add_trace(go.Scatter(
    x=time_course['time'],
    y=time_course['mean_smooth'],
    mode='lines',
    name='Proportion Target Looking',
    line=dict(color=COLORS['primary'], width=3),
    hovertemplate='<b>Time:</b> %{x}ms<br><b>Prop. Target:</b> %{y:.3f}<extra></extra>'
))

# Reference lines
fig.add_hline(y=0.5, line_dash="dash", line_color="gray",
              annotation_text="Chance (0.5)", annotation_position="bottom right")
fig.add_vline(x=0, line_dash="dot", line_color=COLORS['accent'],
              annotation_text="Word Onset", annotation_position="top")
fig.add_vline(x=300, line_dash="dot", line_color=COLORS['warning'], opacity=0.7,
              annotation_text="~300ms", annotation_position="top")

# Add annotation for peak
peak_idx = time_course['mean_smooth'].idxmax()
peak_time = time_course.loc[peak_idx, 'time']
peak_val = time_course.loc[peak_idx, 'mean_smooth']

fig.add_annotation(
    x=peak_time, y=peak_val,
    text=f"Peak: {peak_val:.2f} at {peak_time:.0f}ms",
    showarrow=True,
    arrowhead=2,
    arrowcolor=COLORS['dark'],
    ax=60, ay=-40,
    font=dict(size=12, color=COLORS['dark']),
    bgcolor='white',
    bordercolor=COLORS['dark'],
    borderwidth=1
)

fig.update_layout(
    title=dict(
        text='<b>Time Course of Word Recognition</b><br><sup>Looking-While-Listening Paradigm | Peekbank Database</sup>',
        x=0.5,
        font=dict(size=20)
    ),
    xaxis_title='Time relative to target word onset (ms)',
    yaxis_title='Proportion looking to target',
    xaxis=dict(range=[-500, 2000], dtick=500, gridcolor='rgba(0,0,0,0.1)'),
    yaxis=dict(range=[0.35, 0.75], dtick=0.1, gridcolor='rgba(0,0,0,0.1)'),
    legend=dict(x=0.02, y=0.98, bgcolor='rgba(255,255,255,0.8)'),
    hovermode='x unified',
    template='plotly_white',
    width=1300,
    height=600,
    margin=dict(t=100)
)

fig.show()
fig.write_html('peekbank_data/plot_01_time_course.html')
fig.write_image('peekbank_data/plot_01_time_course.png', scale=2)

## 2. Developmental Trajectory - Animated

In [9]:
# Compute time course by age group
age_tc = tc_data[tc_data['age_group'].notna()].groupby(
    ['age_group', 't_norm']
)['prop_target'].mean().reset_index()

# Smooth each age group
smoothed_data = []
for age in AGE_COLORS.keys():
    age_data = age_tc[age_tc['age_group'] == age].sort_values('t_norm')
    if len(age_data) > 10:
        age_data = age_data.copy()
        age_data['prop_smooth'] = gaussian_filter1d(age_data['prop_target'].values, sigma=2)
        smoothed_data.append(age_data)

age_tc_smooth = pd.concat(smoothed_data)

# Create figure
fig = go.Figure()

for age, color in AGE_COLORS.items():
    data = age_tc_smooth[age_tc_smooth['age_group'] == age]
    if len(data) > 0:
        fig.add_trace(go.Scatter(
            x=data['t_norm'],
            y=data['prop_smooth'],
            mode='lines',
            name=age,
            line=dict(color=color, width=3),
            hovertemplate=f'<b>{age}</b><br>Time: %{{x}}ms<br>Prop: %{{y:.3f}}<extra></extra>'
        ))

fig.add_hline(y=0.5, line_dash="dash", line_color="gray", opacity=0.5)
fig.add_vline(x=0, line_dash="dot", line_color="gray", opacity=0.5)

fig.update_layout(
    title=dict(
        text='<b>Developmental Changes in Word Recognition</b><br><sup>Older children show faster and more accurate word recognition</sup>',
        x=0.5,
        font=dict(size=20)
    ),
    xaxis_title='Time relative to target word onset (ms)',
    yaxis_title='Proportion looking to target',
    xaxis=dict(range=[-500, 2000], gridcolor='rgba(0,0,0,0.1)'),
    yaxis=dict(range=[0.15, 0.99], gridcolor='rgba(0,0,0,0.1)'),
    legend=dict(
        title='<b>Age Group</b>',
        x=1.02, y=1.2,
        bgcolor='rgba(255,255,255,0.9)',
        bordercolor='lightgray',
        borderwidth=1
    ),
    hovermode='x unified',
    template='plotly_white',
    width=1300,
    height=600
)

fig.show()
fig.write_html('peekbank_data/plot_02_age_effects.html')
fig.write_image('peekbank_data/plot_02_age_effects.png', scale=2)

## 3. Age Distribution - Interactive Histogram

In [10]:
ages = administrations['age'].dropna()

fig = go.Figure()

fig.add_trace(go.Histogram(
    x=ages,
    nbinsx=40,
    marker=dict(
        color=ages,
        colorscale='Viridis',
        line=dict(color='white', width=1)
    ),
    hovertemplate='<b>Age:</b> %{x:.0f} months<br><b>Count:</b> %{y}<extra></extra>'
))

# Add mean and median lines
fig.add_vline(x=ages.mean(), line_dash="dash", line_color=COLORS['accent'], line_width=2,
              annotation_text=f"Mean: {ages.mean():.1f}mo", annotation_position="top")
fig.add_vline(x=ages.median(), line_dash="dash", line_color=COLORS['secondary'], line_width=2,
              annotation_text=f"Median: {ages.median():.1f}mo", annotation_position="top left")

fig.update_layout(
    title=dict(
        text='<b>Age Distribution of Children in Peekbank</b><br><sup>N = {:,} testing sessions</sup>'.format(len(ages)),
        x=0.5,
        font=dict(size=20)
    ),
    xaxis_title='Age (months)',
    yaxis_title='Number of Sessions',
    bargap=0.05,
    template='plotly_white',
    width=1300,
    height=600
)

fig.show()
fig.write_html('peekbank_data/plot_03_age_distribution.html')
fig.write_image('peekbank_data/plot_03_age_distribution.png', scale=2)

## 4. Word Recognition Accuracy by Age - Lollipop Chart

In [11]:
# Calculate accuracy by age group
analysis_data = looking_data[
    (looking_data['t_norm'] >= 300) &
    (looking_data['t_norm'] <= 2000)
].copy()

age_accuracy = analysis_data.groupby('age_group').agg({
    'prop_target': ['mean', 'std', 'count']
}).reset_index()
age_accuracy.columns = ['age_group', 'accuracy', 'std', 'n']
age_accuracy['se'] = age_accuracy['std'] / np.sqrt(age_accuracy['n'])
age_accuracy = age_accuracy.dropna().sort_values('age_group')

# Create lollipop chart
fig = go.Figure()

# Stems
for i, row in age_accuracy.iterrows():
    color = AGE_COLORS.get(row['age_group'], COLORS['primary'])
    fig.add_trace(go.Scatter(
        x=[0.5, row['accuracy']],
        y=[row['age_group'], row['age_group']],
        mode='lines',
        line=dict(color=color, width=3),
        showlegend=False,
        hoverinfo='skip'
    ))

# Dots with error bars
fig.add_trace(go.Scatter(
    x=age_accuracy['accuracy'],
    y=age_accuracy['age_group'],
    mode='markers',
    marker=dict(
        size=20,
        color=[AGE_COLORS.get(ag, COLORS['primary']) for ag in age_accuracy['age_group']],
        line=dict(color='white', width=2)
    ),
    error_x=dict(
        type='data',
        array=1.96 * age_accuracy['se'],
        thickness=2,
        width=6,
        color='rgba(0,0,0,0.5)'
    ),
    hovertemplate='<b>%{y}</b><br>Accuracy: %{x:.3f}<extra></extra>',
    showlegend=False
))

# Add value labels
for i, row in age_accuracy.iterrows():
    fig.add_annotation(
        x=row['accuracy'] + 0.02,
        y=row['age_group'],
        text=f"<b>{row['accuracy']:.3f}</b>",
        showarrow=False,
        font=dict(size=12)
    )

# Chance line
fig.add_vline(x=0.5, line_dash="dash", line_color=COLORS['accent'], line_width=2,
              annotation_text="Chance", annotation_position="top")

fig.update_layout(
    title=dict(
        text='<b>Word Recognition Accuracy by Age Group</b><br><sup>Analysis window: 300-2000ms | Error bars: 95% CI</sup>',
        x=0.5,
        font=dict(size=20)
    ),
    xaxis_title='Proportion Looking to Target',
    yaxis_title='',
    xaxis=dict(range=[0.45, 0.75], gridcolor='rgba(0,0,0,0.1)'),
    template='plotly_white',
    width=1300,
    height=600
)

fig.show()
fig.write_html('peekbank_data/plot_04_accuracy_by_age.html')
fig.write_image('peekbank_data/plot_04_accuracy_by_age.png', scale=2)

## 5. Target vs Distractor Competition - Area Chart

In [12]:
# Compute competition dynamics
competition = tc_data.groupby('t_norm').agg({
    'target_looking': 'mean',
    'distractor_looking': 'mean'
}).reset_index()

# Smooth
competition['target_smooth'] = gaussian_filter1d(competition['target_looking'], sigma=2)
competition['distractor_smooth'] = gaussian_filter1d(competition['distractor_looking'], sigma=2)

fig = go.Figure()

# Target area
fig.add_trace(go.Scatter(
    x=competition['t_norm'],
    y=competition['target_smooth'],
    mode='lines',
    name='Target',
    fill='tozeroy',
    fillcolor='rgba(46, 204, 113, 0.4)',
    line=dict(color=COLORS['secondary'], width=3),
    hovertemplate='Target: %{y:.3f}<extra></extra>'
))

# Distractor area
fig.add_trace(go.Scatter(
    x=competition['t_norm'],
    y=competition['distractor_smooth'],
    mode='lines',
    name='Distractor',
    fill='tozeroy',
    fillcolor='rgba(231, 76, 60, 0.4)',
    line=dict(color=COLORS['accent'], width=3),
    hovertemplate='Distractor: %{y:.3f}<extra></extra>'
))

# Find crossover
competition['advantage'] = competition['target_smooth'] - competition['distractor_smooth']
crossover_idx = (competition['advantage'] > 0) & (competition['t_norm'] > 0)
if crossover_idx.any():
    crossover_time = competition[crossover_idx]['t_norm'].iloc[0]
    fig.add_vline(x=crossover_time, line_dash="dash", line_color=COLORS['purple'], line_width=2,
                  annotation_text=f"Crossover: {crossover_time}ms", annotation_position="top")

fig.add_vline(x=0, line_dash="dot", line_color="gray", opacity=0.7,
              annotation_text="Word Onset")

fig.update_layout(
    title=dict(
        text='<b>Target vs Distractor Competition</b><br><sup>How looking behavior shifts after hearing the target word</sup>',
        x=0.5,
        font=dict(size=20)
    ),
    xaxis_title='Time relative to word onset (ms)',
    yaxis_title='Proportion Looking',
    xaxis=dict(range=[-500, 2000], gridcolor='rgba(0,0,0,0.1)'),
    yaxis=dict(range=[0, 0.65], gridcolor='rgba(0,0,0,0.1)'),
    legend=dict(x=0.02, y=0.98, bgcolor='rgba(255,255,255,0.8)'),
    hovermode='x unified',
    template='plotly_white',
    width=1300,
    height=600
)

fig.show()
fig.write_html('peekbank_data/plot_05_competition.html')
fig.write_image('peekbank_data/plot_05_competition.png', scale=2)

## 6. Word-Level Analysis - Interactive Bar Race

In [13]:
# Word accuracy
word_data = analysis_data[analysis_data['target_word'].notna()]
word_accuracy = word_data.groupby('target_word').agg({
    'prop_target': ['mean', 'count']
}).reset_index()
word_accuracy.columns = ['word', 'accuracy', 'n']
word_accuracy = word_accuracy[word_accuracy['n'] >= 500].sort_values('accuracy', ascending=True)

# Get top 15 and bottom 15
top_words = word_accuracy.tail(15)
bottom_words = word_accuracy.head(15)

# Create subplot
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('<b>🏆 Easiest Words</b>', '<b>🎯 Hardest Words</b>'),
    horizontal_spacing=0.2
)

# Easiest words (top)
fig.add_trace(go.Bar(
    y=top_words['word'],
    x=top_words['accuracy'],
    orientation='h',
    marker=dict(
        color=top_words['accuracy'],
        colorscale='Greens',
        line=dict(color='white', width=1)
    ),
    text=[f"{x:.3f}" for x in top_words['accuracy']],
    textposition='outside',
    hovertemplate='<b>%{y}</b><br>Accuracy: %{x:.3f}<extra></extra>',
    showlegend=False
), row=1, col=1)

# Hardest words (bottom)
fig.add_trace(go.Bar(
    y=bottom_words['word'],
    x=bottom_words['accuracy'],
    orientation='h',
    marker=dict(
        color=bottom_words['accuracy'],
        colorscale='Reds_r',
        line=dict(color='white', width=1)
    ),
    text=[f"{x:.3f}" for x in bottom_words['accuracy']],
    textposition='outside',
    hovertemplate='<b>%{y}</b><br>Accuracy: %{x:.3f}<extra></extra>',
    showlegend=False
), row=1, col=2)

# Add chance lines
fig.add_vline(x=0.5, line_dash="dash", line_color="gray", row=1, col=1)
fig.add_vline(x=0.5, line_dash="dash", line_color="gray", row=1, col=2)

fig.update_layout(
    title=dict(
        text='<b>Which Words Are Easiest/Hardest to Recognize?</b><br><sup>Based on {:,} words with ≥500 observations each</sup>'.format(len(word_accuracy)),
        x=0.5,
        font=dict(size=20)
    ),
    template='plotly_white',
    width=1300,
    height=600,
    showlegend=False
)

fig.update_xaxes(title_text='Accuracy', range=[0.4, 1.05], row=1, col=1)
fig.update_xaxes(title_text='Accuracy', range=[0.4, 1.05], row=1, col=2)

fig.show()
fig.write_html('peekbank_data/plot_06_word_accuracy.html')
fig.write_image('peekbank_data/plot_06_word_accuracy.png', scale=2)

## 7. Individual Differences - Scatter with Density

In [14]:
# Subject-level accuracy
subject_accuracy = analysis_data.groupby(['subject_id', 'age']).agg({
    'prop_target': ['mean', 'count']
}).reset_index()
subject_accuracy.columns = ['subject_id', 'age', 'accuracy', 'n']
subject_accuracy = subject_accuracy[subject_accuracy['n'] >= 50]

# Calculate correlation
r, p = stats.pearsonr(subject_accuracy['age'], subject_accuracy['accuracy'])

# Create scatter with marginal distributions
fig = make_subplots(
    rows=2, cols=2,
    specs=[[{"type": "histogram"}, None],
           [{"type": "scatter"}, {"type": "histogram"}]],
    column_widths=[0.85, 0.15],
    row_heights=[0.15, 0.85],
    horizontal_spacing=0.02,
    vertical_spacing=0.02
)

# Main scatter
fig.add_trace(go.Scatter(
    x=subject_accuracy['age'],
    y=subject_accuracy['accuracy'],
    mode='markers',
    marker=dict(
        size=20,
        color=subject_accuracy['age'],
        colorscale='Viridis',
        opacity=0.6,
        line=dict(color='white', width=0.5)
    ),
    hovertemplate='Age: %{x:.1f}mo<br>Accuracy: %{y:.3f}<extra></extra>',
    showlegend=False
), row=2, col=1)

# Regression line
z = np.polyfit(subject_accuracy['age'], subject_accuracy['accuracy'], 1)
p_line = np.poly1d(z)
x_line = np.linspace(subject_accuracy['age'].min(), subject_accuracy['age'].max(), 100)

fig.add_trace(go.Scatter(
    x=x_line,
    y=p_line(x_line),
    mode='lines',
    line=dict(color=COLORS['accent'], width=3),
    name=f'r = {r:.3f}',
    showlegend=True
), row=2, col=1)

# Chance line
fig.add_hline(y=0.5, line_dash="dash", line_color="gray", opacity=0.5, row=2, col=1)

# Top histogram (age)
fig.add_trace(go.Histogram(
    x=subject_accuracy['age'],
    marker=dict(color=COLORS['primary'], line=dict(color='white', width=1)),
    showlegend=False
), row=1, col=1)

# Right histogram (accuracy)
fig.add_trace(go.Histogram(
    y=subject_accuracy['accuracy'],
    marker=dict(color=COLORS['secondary'], line=dict(color='white', width=1)),
    showlegend=False
), row=2, col=2)

fig.update_layout(
    title=dict(
        text='<b>Individual Differences in Word Recognition</b><br><sup>N = {:,} children | Correlation: r = {:.3f}</sup>'.format(len(subject_accuracy), r),
        x=0.5,
        font=dict(size=20)
    ),
    template='plotly_white',
    width=1300,
    height=800,
    legend=dict(x=0.3, y=0.15, bgcolor='rgba(255,255,255,0.8)')
)

fig.update_xaxes(title_text='Age (months)', row=2, col=1)
fig.update_yaxes(title_text='Accuracy', row=2, col=1)
fig.update_xaxes(showticklabels=False, row=1, col=1)
fig.update_yaxes(showticklabels=False, row=2, col=2)

fig.show()
fig.write_html('peekbank_data/plot_07_individual_differences.html')
fig.write_image('peekbank_data/plot_07_individual_differences.png', scale=2)

## 8. Dataset Comparison - Bubble Chart

In [15]:
# Dataset-level statistics
dataset_stats = analysis_data.groupby('dataset_id').agg({
    'prop_target': 'mean',
    'age': ['mean', 'std'],
    'subject_id': 'nunique'
}).reset_index()
dataset_stats.columns = ['dataset_id', 'accuracy', 'mean_age', 'age_sd', 'n_subjects']

dataset_stats = dataset_stats.merge(
    datasets[['dataset_id', 'dataset_name', 'shortcite']],
    on='dataset_id', how='left'
)

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=dataset_stats['mean_age'],
    y=dataset_stats['accuracy'],
    mode='markers',
    marker=dict(
        size=dataset_stats['n_subjects'] * 1.5,
        sizemin=10,
        color=dataset_stats['accuracy'],
        colorscale='RdYlGn',
        colorbar=dict(title='Accuracy'),
        line=dict(color='white', width=2),
        opacity=0.8
    ),
    text=dataset_stats['dataset_name'],
    hovertemplate='<b>%{text}</b><br>Mean Age: %{x:.1f}mo<br>Accuracy: %{y:.3f}<br>N subjects: %{marker.size:.0f}<extra></extra>'
))

# Chance line
fig.add_hline(y=0.5, line_dash="dash", line_color="gray", opacity=0.5,
              annotation_text="Chance", annotation_position="bottom right")

# Add regression line
z = np.polyfit(dataset_stats['mean_age'], dataset_stats['accuracy'], 1)
p_line = np.poly1d(z)
x_line = np.linspace(dataset_stats['mean_age'].min(), dataset_stats['mean_age'].max(), 100)

fig.add_trace(go.Scatter(
    x=x_line,
    y=p_line(x_line),
    mode='lines',
    line=dict(color=COLORS['dark'], width=2, dash='dot'),
    name='Trend',
    showlegend=True
))

fig.update_layout(
    title=dict(
        text='<b>Cross-Dataset Comparison</b><br><sup>Bubble size = number of subjects | Color = accuracy</sup>',
        x=0.5,
        font=dict(size=20)
    ),
    xaxis_title='Mean Age (months)',
    yaxis_title='Mean Accuracy',
    xaxis=dict(gridcolor='rgba(0,0,0,0.1)'),
    yaxis=dict(range=[0.45, 0.85], gridcolor='rgba(0,0,0,0.1)'),
    template='plotly_white',
    width=1300,
    height=600,
    legend=dict(x=0.02, y=0.98)
)

fig.show()
fig.write_html('peekbank_data/plot_08_dataset_comparison.html')
fig.write_image('peekbank_data/plot_08_dataset_comparison.png', scale=2)

## 9. Heatmap: Accuracy by Age and Time

In [16]:
# Create binned data for heatmap
heatmap_data = tc_data.copy()
heatmap_data['age_bin'] = pd.cut(heatmap_data['age'], bins=range(6, 60, 6))
heatmap_data['time_bin'] = pd.cut(heatmap_data['t_norm'], bins=range(-500, 2100, 100))

# Aggregate
heatmap_agg = heatmap_data.groupby(['age_bin', 'time_bin'])['prop_target'].mean().reset_index()
heatmap_pivot = heatmap_agg.pivot(index='age_bin', columns='time_bin', values='prop_target')

# Clean labels
x_labels = [f"{int(x.left)}" for x in heatmap_pivot.columns if pd.notna(x)]
y_labels = [f"{int(y.left)}-{int(y.right)}mo" for y in heatmap_pivot.index if pd.notna(y)]

fig = go.Figure(data=go.Heatmap(
    z=heatmap_pivot.values,
    x=x_labels,
    y=y_labels,
    colorscale='RdBu',
    zmid=0.5,
    colorbar=dict(title='Prop. Target'),
    hovertemplate='Age: %{y}<br>Time: %{x}ms<br>Accuracy: %{z:.3f}<extra></extra>'
))

# Add word onset line
fig.add_vline(x=5, line_dash="solid", line_color="white", line_width=3)

fig.update_layout(
    title=dict(
        text='<b>Word Recognition: Age × Time Interaction</b><br><sup>Heatmap showing accuracy across age groups and time points</sup>',
        x=0.5,
        font=dict(size=20)
    ),
    xaxis_title='Time relative to word onset (ms)',
    yaxis_title='Age Group',
    template='plotly_white',
    width=1300,
    height=600
)

fig.show()
fig.write_html('peekbank_data/plot_09_heatmap.html')
fig.write_image('peekbank_data/plot_09_heatmap.png', scale=2)

## 10. Summary Dashboard

In [17]:
# Create a summary dashboard with key metrics
overall_acc = analysis_data['prop_target'].mean()
n_subjects = looking_data['subject_id'].nunique()
n_datasets = looking_data['dataset_id'].nunique()
age_range = f"{looking_data['age'].min():.0f}-{looking_data['age'].max():.0f}"

fig = make_subplots(
    rows=2, cols=3,
    specs=[
        [{"type": "indicator"}, {"type": "indicator"}, {"type": "indicator"}],
        [{"type": "indicator"}, {"type": "indicator"}, {"type": "indicator"}]
    ],
    vertical_spacing=0.3,
    horizontal_spacing=0.1
)

# Key metrics as indicators
metrics = [
    ("Overall Accuracy", overall_acc, ".3f", "#2ECC71"),
    ("Total Subjects", n_subjects, ",d", "#3498DB"),
    ("Datasets", n_datasets, "d", "#9B59B6"),
    ("Age Range (months)", age_range, "s", "#E67E22"),
    ("Total Timepoints", len(looking_data), ",d", "#1ABC9C"),
    ("Above Chance", "Yes ✓" if overall_acc > 0.5 else "No", "s", "#E74C3C")
]

positions = [(1,1), (1,2), (1,3), (2,1), (2,2), (2,3)]

for (title, value, fmt, color), (row, col) in zip(metrics, positions):
    if fmt == 's':
        number_fmt = {"font": {"size": 40, "color": color}}
        val = value
    else:
        number_fmt = {"font": {"size": 50, "color": color}, "valueformat": fmt}
        val = value

    fig.add_trace(go.Indicator(
        mode="number",
        value=val if not isinstance(val, str) else None,
        number=number_fmt if not isinstance(val, str) else {"font": {"size": 40, "color": color}, "suffix": ""},
        title={"text": f"<b>{title}</b>", "font": {"size": 16}},
    ), row=row, col=col)

    if isinstance(val, str):
        fig.add_annotation(
            text=f"<b>{val}</b>",
            x=(col-1)/3 + 1/6,
            y=1 - (row-1)/2 - 0.25,
            xref="paper", yref="paper",
            font=dict(size=40, color=color),
            showarrow=False
        )

fig.update_layout(
    title=dict(
        text='<b>📊 Peekbank Analysis Summary</b>',
        x=0.5,
        font=dict(size=24)
    ),
    template='plotly_white',
    width=1300,
    height=600,
    #paper_bgcolor='rgba(0,0,0,0)',
    #plot_bgcolor='rgba(0,0,0,0)'
)

fig.show()
fig.write_html('peekbank_data/plot_10_summary.html')
fig.write_image('peekbank_data/plot_10_summary.png', scale=2)

---

## 📁 Export and Summary

In [18]:
# Save data files
datasets.to_csv('peekbank_data/datasets.csv', index=False)
subjects.to_csv('peekbank_data/subjects.csv', index=False)
stimuli.to_csv('peekbank_data/stimuli.csv', index=False)
administrations.to_csv('peekbank_data/administrations.csv', index=False)
word_accuracy.to_csv('peekbank_data/word_accuracy.csv', index=False)
subject_accuracy.to_csv('peekbank_data/subject_accuracy.csv', index=False)
dataset_stats.to_csv('peekbank_data/dataset_stats.csv', index=False)

# Close connection
conn.close()

# List all files
import os
files = sorted(os.listdir('peekbank_data'))

print("="*70)
print("✅ ANALYSIS COMPLETE!")
print("="*70)
print("\n📁 Generated Files:")
print("-"*50)

html_files = [f for f in files if f.endswith('.html')]
png_files = [f for f in files if f.endswith('.png')]
csv_files = [f for f in files if f.endswith('.csv')]

print("\n🎨 Interactive Plots (HTML):")
for f in html_files:
    print(f"   • {f}")

print("\n🖼️ Static Images (PNG):")
for f in png_files:
    print(f"   • {f}")

print("\n📊 Data Files (CSV):")
for f in csv_files:
    print(f"   • {f}")

print("\n" + "="*70)
print("💡 TIP: Download the HTML files for interactive visualizations!")
print("="*70)

✅ ANALYSIS COMPLETE!

📁 Generated Files:
--------------------------------------------------

🎨 Interactive Plots (HTML):
   • plot_01_time_course.html
   • plot_02_age_effects.html
   • plot_03_age_distribution.html
   • plot_04_accuracy_by_age.html
   • plot_05_competition.html
   • plot_06_word_accuracy.html
   • plot_07_individual_differences.html
   • plot_08_dataset_comparison.html
   • plot_09_heatmap.html
   • plot_10_summary.html

🖼️ Static Images (PNG):
   • plot_01_time_course.png
   • plot_02_age_effects.png
   • plot_03_age_distribution.png
   • plot_04_accuracy_by_age.png
   • plot_05_competition.png
   • plot_06_word_accuracy.png
   • plot_07_individual_differences.png
   • plot_08_dataset_comparison.png
   • plot_09_heatmap.png
   • plot_10_summary.png

📊 Data Files (CSV):
   • administrations.csv
   • dataset_stats.csv
   • datasets.csv
   • stimuli.csv
   • subject_accuracy.csv
   • subjects.csv
   • word_accuracy.csv

💡 TIP: Download the HTML files for interactive vis

In [19]:
# Zip all outputs for easy download
import shutil
shutil.make_archive('peekbank_outputs', 'zip', 'peekbank_data')
print("\n📦 Created peekbank_outputs.zip - download this for all files!")


📦 Created peekbank_outputs.zip - download this for all files!


In [20]:
# Zip all outputs for easy download
import shutil
shutil.make_archive('peekbank_datasets', 'zip', 'exported_data')
print("\n📦 Created peekbank_datasets.zip - download this for all files!")


📦 Created peekbank_datasets.zip - download this for all files!
